In [107]:
# Install TensorFlow if not already installed
!pip install tensorflow pandas numpy
!pip install patsy


In [108]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import pandas as pd
from patsy import dmatrix

In [110]:
import pandas as pd
import tensorflow as tf
from patsy import dmatrix

def get_tensors(
    df, 
    long=["serBilir"],       # Longitudinal outcome
    base=["age", "sex", "drug"],  # Baseline covariates
    obstime="year",          # Observation time column
    spline_degree=3,         # Degree for natural spline
    df_spline=5              # Degrees of freedom for spline
):
    """
    Convert DataFrame to TensorFlow tensors for Transformer model.

    Parameters
    ----------
    df : Pandas DataFrame
        Input data with longitudinal, baseline covariates, and observation times.
    long : list of str
        Columns for longitudinal outcomes.
    base : list of str
        Columns for baseline covariates.
    obstime : str
        Column for observation time.
    spline_degree : int
        Degree of the natural spline for the time-varying component.
    df_spline : int
        Degrees of freedom for the natural spline.

    Returns
    -------
    Tensors for Transformer input: x_long, x_base, mask, event_tensor, time_tensor, obs_time
    """
    # Convert categorical columns to numerical encodings
    if "sex" in df:
        df["sex"] = df["sex"].astype("category").cat.codes
    if "drug" in df:
        df["drug"] = df["drug"].astype("category").cat.codes

    # Add subject indices
    df["id_new"] = df.groupby("id").ngroup()

    # Generate natural spline features for `obstime`
    spline_features = dmatrix(
        f"bs({obstime}, df={df_spline}, degree={spline_degree}, include_intercept=False)", 
        data=df, 
        return_type="dataframe"
    )

    # Add spline features to the DataFrame
    spline_columns = [f"spline_{i}" for i in range(spline_features.shape[1])]
    df[spline_columns] = spline_features

    # Update baseline covariates to include spline features
    base += spline_columns

    # Determine tensor dimensions
    num_subjects = df["id_new"].nunique()
    max_visits = df.groupby("id")["year"].count().max()

    # Initialize tensors
    x_base = tf.zeros((num_subjects, max_visits, len(base)), dtype=tf.float32)
    x_long = tf.zeros((num_subjects, max_visits, len(long)), dtype=tf.float32)
    mask = tf.zeros((num_subjects, max_visits), dtype=tf.bool)
    obs_time = tf.zeros((num_subjects, max_visits), dtype=tf.float32)

    # Populate tensors
    for _, row in df.iterrows():
        subj_idx = int(row["id_new"])
        visit_idx = int(row["year"])
        
        # Populate baseline covariates
        x_base = tf.tensor_scatter_nd_update(
            x_base, [[subj_idx, visit_idx]], [row[base].values]
        )
        
        # Populate longitudinal outcomes
        x_long = tf.tensor_scatter_nd_update(
            x_long, [[subj_idx, visit_idx]], [row[long].values]
        )
        
        # Populate observation time and mask
        mask = tf.tensor_scatter_nd_update(mask, [[subj_idx, visit_idx]], [True])
        obs_time = tf.tensor_scatter_nd_update(
            obs_time, [[subj_idx, visit_idx]], [row[obstime]]
        )

    # Extract event and time tensors
    event_tensor = tf.convert_to_tensor(
        df[df["year"] == 0]["status2"].values, dtype=tf.float32
    )
    time_tensor = tf.convert_to_tensor(
        df[df["year"] == 0]["years"].values, dtype=tf.float32
    )

    return x_long, x_base, mask, event_tensor, time_tensor, obs_time


In [111]:
import pandas as pd
import tensorflow as tf
from patsy import dmatrix

# Load dataset
df = pd.read_csv("pbc2_dataset.csv")  # Replace with the correct path to your dataset

# Call the function
x_long, x_base, mask, event_tensor, time_tensor, obs_time = get_tensors(
    df,
    long=["serBilir"],       # Longitudinal outcome
    base=["age", "sex", "drug"],  # Baseline covariates
    obstime="year",          # Observation time column
    spline_degree=3,      # Degree for natural spline
     df_spline=3   
)

# View outputs
print("Longitudinal Covariates Tensor (x_long):")
print(x_long.numpy())

print("\nBaseline Covariates Tensor (x_base):")
print(x_base.numpy())

print("\nMask Tensor (mask):")
print(mask.numpy())

print("\nEvent Indicator Tensor (event_tensor):")
print(event_tensor.numpy())

print("\nEvent Time Tensor (time_tensor):")
print(time_tensor.numpy())

print("\nObservation Time Tensor (obs_time):")
print(obs_time.numpy())


Longitudinal Covariates Tensor (x_long):
[[[21.3]
  [ 0. ]
  [ 0. ]
  ...
  [ 0. ]
  [ 0. ]
  [ 0. ]]

 [[ 1. ]
  [ 0. ]
  [ 1.9]
  ...
  [ 0. ]
  [ 0. ]
  [ 0. ]]

 [[ 1.5]
  [ 0. ]
  [ 1.8]
  ...
  [ 0. ]
  [ 0. ]
  [ 0. ]]

 ...

 [[ 1.7]
  [ 2.2]
  [ 1.4]
  ...
  [ 0. ]
  [ 0. ]
  [ 0. ]]

 [[ 1.5]
  [ 1.9]
  [ 0. ]
  ...
  [ 0. ]
  [ 0. ]
  [ 0. ]]

 [[ 5.5]
  [ 7.4]
  [23.4]
  ...
  [ 0. ]
  [ 0. ]
  [ 0. ]]]

Baseline Covariates Tensor (x_base):
[[[5.87668381e+01 0.00000000e+00 0.00000000e+00 ... 1.03623502e-01
   4.01123241e-03 5.17578374e-05]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00

In [112]:
def get_mask(pad, future=True, window=None):
    """
    Create attention masks for padding and future lookahead.

    Parameters
    ----------
    pad : Tensor
        Padding tensor indicating non-padded elements.
    future : bool
        If True, prevent attention to future time steps.
    window : int
        Optional sliding window for attention.

    Returns
    -------
    mask : Tensor
        Attention mask for the Transformer.
    """
    size = tf.shape(pad)[1]
    mask = tf.expand_dims(tf.cast(pad != 0, tf.float32), axis=1)  # Padding mask

    if future:
        future_mask = tf.linalg.band_part(tf.ones((size, size)), 0, -1)
        if window is not None:
            window_mask = tf.linalg.band_part(tf.ones((size, size)), -window, 0)
            future_mask *= window_mask
        mask = mask * tf.expand_dims(future_mask, axis=0)

    # Expand mask to align with multi-head attention shape
    return tf.expand_dims(mask, axis=1)  # (batch_size, 1, seq_len, seq_len)


In [120]:
class NoamOpt(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, model_size, warmup_steps, factor=1):
        self.model_size = tf.cast(model_size, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)
        self.factor = factor

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        rate = self.factor * (self.model_size ** -0.5) * tf.minimum(
            step ** -0.5, step * (self.warmup_steps ** -1.5)
        )
        return rate


In [121]:


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, nhead, dropout=0.1):
        """
        Multi-Head Attention layer for Transformer.

        Parameters:
        ----------
        d_model: int
            Total dimension of the model (embedding size).
        nhead: int
            Number of attention heads.
        dropout: float
            Dropout rate.
        """
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.d_k = d_model // nhead

        assert d_model % nhead == 0, "d_model must be divisible by nhead"

        # Linear layers for query, key, value transformations
        self.q_linear = tf.keras.layers.Dense(d_model, use_bias=False)
        self.k_linear = tf.keras.layers.Dense(d_model, use_bias=False)
        self.v_linear = tf.keras.layers.Dense(d_model, use_bias=False)
        self.out = tf.keras.layers.Dense(d_model)

        self.dropout = tf.keras.layers.Dropout(dropout)

    def attention(self, query, key, value, d_k, mask=None):
        """
        Scaled Dot-Product Attention.

        Parameters:
        ----------
        query, key, value: Tensors of shape (batch_size, seq_len, d_k)
        d_k: int
            Dimension of key vectors.
        mask: Tensor, optional
            Mask to ignore certain time steps, shape (batch_size, seq_len).

        Returns:
        -------
        output: Tensor
            Attention output.
        """
        scores = tf.matmul(query, key, transpose_b=True) / np.sqrt(d_k)

        # Apply mask if provided
        if mask is not None:
            scores += (mask * -1e9)  # Large negative value to ignore

        # Softmax normalization
        scores = tf.nn.softmax(scores, axis=-1)

        # Apply dropout
        scores = self.dropout(scores)

        # Weighted sum of values
        output = tf.matmul(scores, value)
        return output

    def call(self, query, key, value, mask=None):
        """
        Forward pass for Multi-Head Attention.

        Parameters:
        ----------
        query, key, value: Tensors of shape (batch_size, seq_len, d_model)
        mask: Tensor, optional
            Mask for padding or future steps.

        Returns:
        -------
        output: Tensor
            Multi-head attention output, shape (batch_size, seq_len, d_model).
        """
        batch_size = tf.shape(query)[0]

        # Linear projections
        query = self.q_linear(query)  # (batch_size, seq_len, d_model)
        key = self.k_linear(key)
        value = self.v_linear(value)

        # Split into multiple heads and reshape
        query = tf.reshape(query, (batch_size, -1, self.nhead, self.d_k))  # (batch_size, seq_len, nhead, d_k)
        key = tf.reshape(key, (batch_size, -1, self.nhead, self.d_k))
        value = tf.reshape(value, (batch_size, -1, self.nhead, self.d_k))

        # Transpose to (batch_size, nhead, seq_len, d_k)
        query = tf.transpose(query, perm=[0, 2, 1, 3])
        key = tf.transpose(key, perm=[0, 2, 1, 3])
        value = tf.transpose(value, perm=[0, 2, 1, 3])

        # Apply attention
        scores = self.attention(query, key, value, self.d_k, mask)

        # Concatenate heads and reshape back to (batch_size, seq_len, d_model)
        scores = tf.transpose(scores, perm=[0, 2, 1, 3])  # (batch_size, seq_len, nhead, d_k)
        concat = tf.reshape(scores, (batch_size, -1, self.d_model))

        # Final linear layer
        output = self.out(concat)
        return output


In [122]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout
from tensorflow.keras import Sequential


class DecoderLayer(tf.keras.layers.Layer):
    """
    Transformer Decoder Layer

    Parameters
    ----------
    d_model : int
        Dimension of the input vector (embedding size).
    nhead : int
        Number of attention heads.
    dropout : float
        Dropout rate.
    """
    
    def __init__(self, d_model, nhead, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.dropout = Dropout(dropout)
        
        # Multi-Head Attention
        self.attention = MultiHeadAttention(d_model, nhead, dropout)
        
        # Feed-Forward Network
        self.feed_forward = Sequential([
            Dense(64, activation='relu'),
            Dense(d_model),
            Dropout(dropout)
        ])
        
        # Layer Normalization
        self.layer_norm1 = LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = LayerNormalization(epsilon=1e-6)
    
    def call(self, q, kv, mask, training=False):
        """
        Forward pass for the decoder layer.

        Parameters
        ----------
        q : tf.Tensor
            Query tensor (e.g., target sequence), shape (batch_size, seq_len, d_model).
        kv : tf.Tensor
            Key-Value tensor (e.g., source sequence), shape (batch_size, seq_len, d_model).
        mask : tf.Tensor
            Attention mask.
        training : bool
            If True, applies dropout.

        Returns
        -------
        x : tf.Tensor
            Processed output tensor, shape (batch_size, seq_len, d_model).
        """
        # Attention Block
        residual = q
        x = self.attention(query=q, key=kv, value=kv, mask=mask)  # Multi-Head Attention
        x = self.dropout(x, training=training)
        x = self.layer_norm1(x + residual)  # Add & Norm
        
        # Feed-Forward Block
        residual = x
        x = self.feed_forward(x, training=training)
        x = self.layer_norm2(x + residual)  # Add & Norm
        
        return x


In [123]:
import tensorflow as tf
import numpy as np

def positional_encoding(batch_size, length, d_model, obs_time):
    """
    Positional Encoding for each visit.

    Parameters
    ----------
    batch_size : int
        Number of subjects in batch.
    length : int
        Number of visits.
    d_model : int
        Dimension of the model vector.
    obs_time : tf.Tensor
        Observed/recorded time of each visit, shape (batch_size, length) or (length).

    Returns
    -------
    PE : tf.Tensor
        Positional encodings, shape (batch_size, length, d_model).
    """
    # Ensure obs_time has consistent shape
    if tf.rank(obs_time) == 0:  # Scalar
        obs_time = tf.expand_dims(tf.repeat(obs_time, batch_size), axis=1)
    elif tf.rank(obs_time) == 1:  # 1D tensor
        obs_time = tf.tile(tf.expand_dims(obs_time, axis=0), [batch_size, 1])

    # Initialize positional encoding matrix
    PE = tf.zeros((batch_size, length, d_model), dtype=tf.float32)

    # Compute positional encoding for even indices
    even_indices = tf.cast(tf.range(0, d_model, 2), dtype=tf.float32)  # Cast to float32
    even_angles = tf.pow(10000.0, even_indices / d_model)
    PE_even = tf.sin(tf.einsum('ij,k->ijk', obs_time, 1 / even_angles))

    # Compute positional encoding for odd indices
    odd_indices = tf.cast(tf.range(1, d_model, 2), dtype=tf.float32)  # Cast to float32
    odd_angles = tf.pow(10000.0, odd_indices / d_model)
    PE_odd = tf.cos(tf.einsum('ij,k->ijk', obs_time, 1 / odd_angles))

    # Combine even and odd positional encodings
    PE = tf.concat([PE_even, PE_odd], axis=-1)

    return PE


In [126]:
# Check the data type of a tensor (mask)
if mask.dtype == tf.bool:
    print("The mask is of type bool.")
elif mask.dtype == tf.float32:
    print("The mask is of type float32.")
else:
    print(f"The mask is of type {mask.dtype}.")


The mask is of type bool.


In [127]:
print("Mask dtype:", mask.dtype)
print("Mask shape:", mask.shape)


Mask dtype: <dtype: 'bool'>
Mask shape: (312, 16)


In [128]:
# Configuration for PBC2 dataset
d_long = 1           # Univariate longitudinal outcome
d_base = 3           # Baseline covariates (age, sex, drug, and splines for year)
d_model = 32         # Internal representation dimension
nhead = 4            # 4 attention heads (divides d_model evenly)
num_decoder_layers = 3  # Three stacked decoder layers
dropout = 0.2        # 20% dropout to prevent overfitting


In [129]:

# Decoder Layer
class DecoderLayer(layers.Layer):
    def __init__(self, d_model, nhead, dropout):
        super(DecoderLayer, self).__init__()
        self.mha = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model)
        self.dropout1 = layers.Dropout(dropout)
        self.norm1 = layers.LayerNormalization()
        self.ffn = tf.keras.Sequential([
            layers.Dense(d_model * 4, activation='relu'),
            layers.Dense(d_model)
        ])
        self.dropout2 = layers.Dropout(dropout)
        self.norm2 = layers.LayerNormalization()

    def call(self, q, k, v, mask, training=False):
         # Convert mask to float32 for compatibility
        mask = tf.cast(mask, dtype=tf.float32)
        # Multi-head attention
        attn_output = self.mha(query=q, key=k, value=v, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(q + attn_output)  # Add & Norm

        # Feed-forward network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.norm2(out1 + ffn_output)  # Add & Norm








# Decoder Block
class Decoder(Model):
    def __init__(self, d_long, d_base, d_model, nhead, num_decoder_layers, dropout):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.Sequential([
            layers.Dense(d_model, activation='relu'),
            layers.Dropout(0.2),
            layers.LayerNormalization(),
            layers.Dense(d_model)
        ])
        self.decoder_layers = [DecoderLayer(d_model, nhead, dropout) for _ in range(num_decoder_layers)]

    def call(self, long, base, mask, obs_time, training=False):
        # Concatenate longitudinal and baseline data
        x = tf.concat([long, base], axis=-1)

        # Linear Embedding
        x = self.embedding(x)

        # Positional Embedding
        pos_enc = positional_encoding(tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2], obs_time)
        x += pos_enc
        
         # Convert mask to float32 for compatibility
        mask = tf.cast(mask, dtype=tf.float32)
        
        # Decoder Layers
        for layer in self.decoder_layers:
            x = layer(x, x, x, mask, training=training)
        return x
    


# Prediction Decoder Block
class DecoderP(Model):
    def __init__(self, d_model, nhead, num_decoder_layers, dropout):
        super(DecoderP, self).__init__()
        self.decoder_layers = [DecoderLayer(d_model, nhead, dropout) for _ in range(num_decoder_layers)]

    def call(self, q, kv, mask, pred_time, training=False):
        # Positional Embedding
        pos_enc = positional_encoding(tf.shape(q)[0], tf.shape(q)[1], tf.shape(q)[2], pred_time)
        q += pos_enc

        # Decoder Layers
        for layer in self.decoder_layers:
            q = layer(q, kv, kv, mask, training=training)
        return q

# Full Transformer Model
class Transformer(Model):
    def __init__(self, d_long, d_base, d_model=32, nhead=4, num_decoder_layers=3, dropout=0.2):
        super(Transformer, self).__init__()
        self.decoder = Decoder(d_long, d_base, d_model, nhead, num_decoder_layers, dropout)
        self.decoder_pred = DecoderP(d_model, nhead, 1, dropout)
        self.long_out = layers.Dense(d_long)
        self.surv_out = layers.Dense(1, activation='sigmoid')

    def call(self, long, base, mask, obs_time, pred_time, training=False):
        x = self.decoder(long, base, mask, obs_time, training=training)
        x = self.decoder_pred(x, x, mask, pred_time, training=training)
        long_pred = self.long_out(x)
        surv_pred = self.surv_out(x)
        return long_pred, surv_pred




In [130]:
# Print the shapes of the input tensors
print("Shape of Longitudinal Data (x_long):", x_long.shape)
print("Shape of Baseline Covariates (x_base):", x_base.shape)
print("Shape of Mask (mask):", mask.shape)
print("Shape of Observation Time (obs_time):", obs_time.shape)
print("Shape of Prediction Time (pred_time):", pred_time.shape)


Shape of Longitudinal Data (x_long): (312, 16, 1)
Shape of Baseline Covariates (x_base): (312, 16, 9)
Shape of Mask (mask): (312, 16)
Shape of Observation Time (obs_time): (312, 16)
Shape of Prediction Time (pred_time): (312, 16)


In [131]:
# Generate the input tensors from PBC2 dataset
x_long, x_base, mask, event_tensor, time_tensor, obs_time = get_tensors(df)

# Define prediction times (can reuse `obs_time` for simplicity)
pred_time = obs_time


In [132]:
# Initialize the Transformer model
transformer = Transformer(
    d_long=d_long,
    d_base=d_base,
    d_model=d_model,
    nhead=nhead,
    num_decoder_layers=num_decoder_layers,
    dropout=dropout
)


In [133]:
# Pass inputs through the Transformer model
long_pred, surv_pred = transformer(
    long=x_long,
    base=x_base,
    mask=mask,
    obs_time=obs_time,
    pred_time=pred_time,
    training=False
)


TypeError: Exception encountered when calling Decoder.call().

[1m`x` and `y` must have the same dtype, got tf.float32 != tf.int32.[0m

Arguments received by Decoder.call():
  • long=tf.Tensor(shape=(312, 16, 1), dtype=float32)
  • base=tf.Tensor(shape=(312, 16, 15), dtype=float32)
  • mask=tf.Tensor(shape=(312, 16), dtype=bool)
  • obs_time=tf.Tensor(shape=(312, 16), dtype=float32)
  • training=False

In [134]:
print("x dtype:", x.dtype)
print("y dtype:", y.dtype)


NameError: name 'x' is not defined

In [ ]:
import tensorflow as tf

def long_loss(yhat, y, mask):
    """
    Loss for longitudinal outcomes (mean squared error with masking).

    Parameters
    ----------
    yhat : tf.Tensor
        Predicted longitudinal outcomes, shape (batch_size, seq_len, num_features).
    y : tf.Tensor
        True longitudinal outcomes, shape (batch_size, seq_len, num_features).
    mask : tf.Tensor
        Mask indicating valid data (1 for valid, 0 for padding), shape (batch_size, seq_len).

    Returns
    -------
    loss : tf.Tensor
        Scalar representing the mean loss for valid elements.
    """
    # Expand the mask to match the number of features
    mask = tf.expand_dims(mask, axis=-1)  # Shape: (batch_size, seq_len, 1)
    
    # Compute MSE loss element-wise
    loss = tf.keras.losses.mean_squared_error(y, yhat)  # Shape: (batch_size, seq_len, num_features)
    
    # Apply the mask to exclude padding
    loss *= tf.cast(mask, tf.float32)
    
    # Normalize the loss by the number of valid elements
    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
    return loss


In [ ]:
import numpy as np

def surv_loss(surv_pred, mask, event):
    """
    Loss for survival predictions (negative log-likelihood).

    Parameters
    ----------
    surv_pred : tf.Tensor
        Predicted survival probabilities, shape (batch_size, seq_len).
    mask : tf.Tensor
        Mask indicating valid data (1 for valid, 0 for padding), shape (batch_size, seq_len).
    event : tf.Tensor
        Event indicators (1 if event occurred, 0 otherwise), shape (batch_size).

    Returns
    -------
    loss : tf.Tensor
        Scalar representing the mean negative log-likelihood loss.
    """
    # Convert mask and event tensors to NumPy arrays for indexing
    mask_np = mask.numpy()
    event_np = event.numpy()

    # Exclude the first time step and reverse the mask
    mask_out = mask_np[:, 1:]
    mask_rev = mask_out[:, ::-1]
    
    # Determine the index of the last valid time step (event time index)
    event_time_index = mask_out.shape[1] - np.argmax(mask_rev, axis=1) - 1
    
    # Create event filter (e_filter) and survival filter (s_filter)
    e_filter = np.zeros_like(mask_out)
    for row_idx, event_occurred in enumerate(event_np):
        if event_occurred:
            e_filter[row_idx, event_time_index[row_idx]] = 1
    s_filter = mask_out - e_filter

    # Convert filters to TensorFlow tensors
    s_filter = tf.convert_to_tensor(s_filter, dtype=tf.float32)
    e_filter = tf.convert_to_tensor(e_filter, dtype=tf.float32)
    
    # Calculate negative log-likelihood
    surv_pred = tf.squeeze(surv_pred)  # Remove unnecessary dimensions
    nll_loss = (
        tf.math.log(surv_pred) * s_filter +
        tf.math.log(1 - surv_pred) * e_filter
    )
    
    # Normalize loss by the total number of valid observations
    nll_loss = tf.reduce_sum(nll_loss) / tf.reduce_sum(mask_out)
    return -nll_loss


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score


In [ ]:
def get_integrated(x, times):
    """
    Compute the integral of a time-dependent metric using the trapezoidal rule.

    Parameters
    ----------
    x : np.array
        Metric values at different times.
    times : np.array
        Corresponding time points.

    Returns
    -------
    float
        Integrated metric value normalized by the time range.
    """
    return np.trapz(x, times) / (max(times) - min(times))


In [ ]:
def AUC(surv_probs, event, time, pred_times):
    """
    Compute AUC and integrated AUC for survival probabilities.

    Parameters
    ----------
    surv_probs : np.array
        Predicted survival probabilities (shape: [num_samples, num_pred_times]).
    event : np.array
        Event indicators (1 for event, 0 for censoring).
    time : np.array
        Event or censoring times.
    pred_times : np.array
        Times at which predictions are made.

    Returns
    -------
    auc : np.array
        AUC values at each prediction time.
    iauc : float
        Integrated AUC over the prediction times.
    """
    auc = []
    for t in range(surv_probs.shape[1]):
        valid_indices = time >= pred_times[t]  # Only consider valid events at this prediction time
        if valid_indices.sum() > 1:  # Ensure at least two valid samples
            auc.append(roc_auc_score(event[valid_indices], surv_probs[valid_indices, t]))
        else:
            auc.append(np.nan)  # Undefined AUC if not enough data

    auc = np.array(auc)
    iauc = get_integrated(auc, pred_times)
    return auc, iauc


In [ ]:
def Brier(surv_probs, event, time, event_train, time_train, LT, pred_windows):
    """
    Compute Brier score and integrated Brier score.

    Parameters
    ----------
    surv_probs : np.array
        Predicted survival probabilities (shape: [num_samples, num_pred_times]).
    event : np.array
        Event indicators for test data.
    time : np.array
        Event or censoring times for test data.
    event_train : np.array
        Event indicators for training data.
    time_train : np.array
        Event or censoring times for training data.
    LT : float
        Landmark time.
    pred_windows : np.array
        Prediction time windows.

    Returns
    -------
    bs : np.array
        Brier scores at each prediction window.
    ibs : float
        Integrated Brier score over the prediction windows.
    """
    bs = []
    for t in pred_windows:
        valid_indices = (time >= LT) & (time <= t)
        if valid_indices.sum() > 0:
            y_true = (event[valid_indices] == 1).astype(float)
            y_pred = 1 - surv_probs[valid_indices, np.searchsorted(pred_windows, t)]
            bs.append(np.mean((y_true - y_pred) ** 2))
        else:
            bs.append(np.nan)  # Undefined Brier score if not enough data

    bs = np.array(bs)
    ibs = get_integrated(bs, pred_windows)
    return bs, ibs


In [ ]:
def MSE(y, yhat):
    """
    Compute Mean Squared Error for longitudinal outcomes.

    Parameters
    ----------
    y : np.array
        True values (shape: [num_samples, num_time_steps]).
    yhat : np.array
        Predicted values (shape: [num_samples, num_time_steps]).

    Returns
    -------
    mse : float
        Mean squared error across subjects and time.
    """
    mse = np.square(y - yhat)
    with warnings.catch_warnings():
        warnings.filterwarnings(action='ignore', message='Mean of empty slice')
        mse = np.nanmean(mse, axis=1)  # Average over time
    mse = np.nanmean(mse, axis=0)  # Average over subjects
    return mse
